In [ ]:
# R code for models regressing responses against events *only* as part of Analysis I:
# Understanding the Role of Events

In [2]:
# Load some packages
library(reshape2)
library(ggplot2)
library(cowplot)
library(ez)
library(nlme)
library(multcomp)
library(pastecs)
theme_set(theme_cowplot(font_size=8))
library(usdm)
library(r2glmm)
library(car)
library(MASS)
library(MCMCglmm)
library(lmerTest)
library(expss)
library(moments)

In [11]:
# Read signals
signals = read.csv("../metrics/all_events.csv")
colnames(signals)

# Set up planned contrasts
signals$Event = factor(signals$Event, levels=sort(unique(signals$Event)))
StopvsNoStop <- c(-1, 1, 1, -1)
TurnvsPass <- c(1, 0, 0, -1)
IntervsCarStop <- c(0, 1, -1, 0)
contrasts(signals$Event) <- cbind(StopvsNoStop, TurnvsPass, IntervsCarStop)
attr(signals$Event, "contrasts")
levels(signals$Event)

[1] "pid"          "LatDist"      "LongDist"     "MaxVel"       "MeanVel"     
 [6] "MaxLatAcc"    "MeanLatAcc"   "MaxLongAcc"   "MeanLongAcc"  "MaxLatJerk"  
[11] "MeanLatJerk"  "MaxLongJerk"  "MeanLongJerk" "NumPks"       "MaxPkAmp"    
[16] "MeanScl"      "MeanEntropy"  "MaxEntropy"   "MeanDwell"    "MaxDwell"    
[21] "MeanHr"       "MaxHr"        "Event"        "Trial"

StopvsNoStop,TurnvsPass,IntervsCarStop
-1,1,0
1,0,1
1,0,-1
-1,-1,0


[1] "1" "2" "3" "4"

In [8]:
# column indexes for the response/dependent variables of interest (NumPks, MaxPkAmp, MaxHR --
# MaxEntropy is handled separately due to missing data in signals)
response_ids = c(14, 15, 16, 21, 22)
entropy_response_ids = c(17, 18)
responses = colnames(signals)[response_ids]
entropy_responses = colnames(signals)[entropy_response_ids]

#---------------------------------------- Models ------------------------------------------#
non_entropy_models = lapply(responses, function(x) {
    lme(
        eval(substitute(resp ~ Event, list(resp = as.name(x)))),
        random = ~1|pid/Trial,
        data = signals,
        method = "ML"
    )
})

entropy_models = lapply(entropy_responses, function(x) {
    lme(
        eval(substitute(resp ~ Event, list(resp = as.name(x)))),
        random = ~1|pid/Trial,
        data = na.omit(signals),
        method = "ML"
    )
})

In [9]:
# Combine all models
event_models = c(non_entropy_models, entropy_models)

response_names = c(
    "NUMBER OF PEAKS",
    "MAX PEAK AMPLITUDE",
    "MEAN SCL", 
    "MEAN HR",
    "MAX HR",
    "MEAN ENTROPY",
    "MAX ENTROPY"
)
model_idx  = c(1:7)

In [10]:
cat("ALL MAIN EFFECTS\n")
cat("-------------------------------------------------------------------------------\n")

model_print = function(x, y, z){
    cat(y[x])
    cat("\n")
    print(summary(z[[x]]))
    cat("\n\n\n")
    cat("==============================================================================
    \n\n\n")
}

lapply(model_idx, model_print, y=response_names, z=event_models)

ALL MAIN EFFECTS
-------------------------------------------------------------------------------
NUMBER OF PEAKS
Linear mixed-effects model fit by maximum likelihood
 Data: signals 
       AIC      BIC    logLik
  1017.109 1043.105 -501.5543

Random effects:
 Formula: ~1 | pid
        (Intercept)
StdDev:   0.5198782

 Formula: ~1 | Trial %in% pid
         (Intercept) Residual
StdDev: 0.0001960215 1.212345

Fixed effects: eval(substitute(resp ~ Event, list(resp = as.name(x)))) 
                         Value  Std.Error  DF   t-value p-value
(Intercept)          1.1310837 0.13663947 223  8.277869  0.0000
EventStopvsNoStop    0.4866360 0.07012177 223  6.939871  0.0000
EventTurnvsPass     -0.1937277 0.09903430 223 -1.956168  0.0517
EventIntervsCarStop -0.0337890 0.09933990 223 -0.340135  0.7341
 Correlation: 
                    (Intr) EvnSNS EvntTP
EventStopvsNoStop    0.002              
EventTurnvsPass     -0.006  0.009       
EventIntervsCarStop -0.003 -0.005  0.000

Standardized Withi

[[1]]
NULL

[[2]]
NULL

[[3]]
NULL

[[4]]
NULL

[[5]]
NULL

[[6]]
NULL

[[7]]
NULL